Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from collections import deque
import heapq



# BFS Algorithm with 3x3 puzzle:

In [2]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [3]:

def generate_goal(PUZZLE_DIM):
    """Generate the goal state for an NxN puzzle."""
    numbers = np.arange(1, PUZZLE_DIM * PUZZLE_DIM)  # Create numbers from 1 to (PUZZLE_DIM * PUZZLE_DIM - 1)
    goal = np.append(numbers, 0)  # Append 0 at the end
    return goal.reshape(PUZZLE_DIM, PUZZLE_DIM)  # Return reshaped NxN NumPy array

In [4]:
goal = generate_goal(PUZZLE_DIM)
print(goal)

[[1 2 3]
 [4 5 6]
 [7 8 0]]


In [5]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [6]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

array([[1, 6, 5],
       [3, 0, 2],
       [8, 4, 7]])

In [7]:
def find_zero(state):
    """Find the position of 0 in the grid."""
    return np.argwhere(state == 0)[0]


In [8]:
def get_neighbors(state):
    """Generate valid neighbors by moving the blank tile."""
    neighbors = []
    x, y = find_zero(state)
    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right

    for dx, dy in moves:
        nx, ny = x + dx, y + dy
        if 0 <= nx < state.shape[0] and 0 <= ny < state.shape[1]:  # check bounds
            new_state = state.copy()
            # swap 0 with the neighbor
            new_state[x, y], new_state[nx, ny] = new_state[nx, ny], new_state[x, y]
            neighbors.append(new_state)

    return neighbors

In [9]:
def bfs(start, goal):
    """Breadth-First Search to solve the puzzle with cost calculation."""
    # queue stores tuples: (state, path, cost)
    queue = deque([(start, [], 0)])  # (state, path of moves, cost)
    visited = set()  # to avoid revisiting states
    visited.add(start.tobytes())
    evaluated_count = 0

    while queue:
        current_state, path, cost = queue.popleft()
        evaluated_count += 1

        
        if np.array_equal(current_state, goal):
            return path, cost, evaluated_count  # return both path and total cost
        
        # explore neighbors
        for neighbor in get_neighbors(current_state):
            if neighbor.tobytes() not in visited:
                visited.add(neighbor.tobytes())
                # append neighbor state, path, and increased cost
                queue.append((neighbor, path + [neighbor], cost + 1))
    
    return None, None, evaluated_count  # No solution

In [10]:

solution_path, total_cost, evaluated_cases = bfs(state, goal)
counter = 0
if solution_path is not None:
    print("Solution Path:")
    for step in solution_path:
        counter = counter +1
        print("Step number :", counter)
        print(step)
    print(f"Total cost (number of moves): {total_cost}")
else:
    print("No solution found.")

print(f"Total number of evaluated cases: {evaluated_cases}")


Solution Path:
Step number : 1
[[1 0 5]
 [3 6 2]
 [8 4 7]]
Step number : 2
[[1 5 0]
 [3 6 2]
 [8 4 7]]
Step number : 3
[[1 5 2]
 [3 6 0]
 [8 4 7]]
Step number : 4
[[1 5 2]
 [3 0 6]
 [8 4 7]]
Step number : 5
[[1 5 2]
 [0 3 6]
 [8 4 7]]
Step number : 6
[[1 5 2]
 [8 3 6]
 [0 4 7]]
Step number : 7
[[1 5 2]
 [8 3 6]
 [4 0 7]]
Step number : 8
[[1 5 2]
 [8 3 6]
 [4 7 0]]
Step number : 9
[[1 5 2]
 [8 3 0]
 [4 7 6]]
Step number : 10
[[1 5 2]
 [8 0 3]
 [4 7 6]]
Step number : 11
[[1 5 2]
 [0 8 3]
 [4 7 6]]
Step number : 12
[[1 5 2]
 [4 8 3]
 [0 7 6]]
Step number : 13
[[1 5 2]
 [4 8 3]
 [7 0 6]]
Step number : 14
[[1 5 2]
 [4 0 3]
 [7 8 6]]
Step number : 15
[[1 0 2]
 [4 5 3]
 [7 8 6]]
Step number : 16
[[1 2 0]
 [4 5 3]
 [7 8 6]]
Step number : 17
[[1 2 3]
 [4 5 0]
 [7 8 6]]
Step number : 18
[[1 2 3]
 [4 5 6]
 [7 8 0]]
Total cost (number of moves): 18
Total number of evaluated cases: 24735


# A* Algorithm with 4x4 puzzle

In [21]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])

In [22]:

def generate_goal(PUZZLE_DIM):
    """Generate the goal state for an NxN puzzle."""
    numbers = np.arange(1, PUZZLE_DIM * PUZZLE_DIM)  # create numbers from 1 to (PUZZLE_DIM * PUZZLE_DIM - 1)
    goal = np.append(numbers, 0)  # append 0 at the end
    return goal.reshape(PUZZLE_DIM, PUZZLE_DIM)  # return reshaped NxN NumPy array

In [23]:
goal = generate_goal(PUZZLE_DIM)
print(goal)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]]


In [24]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [25]:
RANDOMIZE_STEPS = 150
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing:   0%|          | 0/150 [00:00<?, ?it/s]

array([[ 9,  1, 11,  7],
       [ 6,  0,  8,  2],
       [ 5, 10, 12,  4],
       [13, 14,  3, 15]])

In [26]:
def find_zero(state):
    """Find the position of 0 in the grid."""
    return np.argwhere(state == 0)[0]


In [27]:
def manhattan_distance(state, goal):
    """Calculate the Manhattan distance between the current state and the goal state."""
    distance = 0
    rows, cols = state.shape
    for r in range(rows):
        for c in range(cols):
            if state[r, c] != 0:  # Ignore the empty space
                goal_pos = np.argwhere(goal == state[r, c])[0]
                distance += abs(goal_pos[0] - r) + abs(goal_pos[1] - c)
    return distance

In [28]:
def get_neighbors(state):
    """Generate possible neighbors for a given state by swapping the empty space (0)."""
    neighbors = []
    rows, cols = state.shape
    empty_space_pos = tuple(np.argwhere(state == 0)[0])  # find the position of 0

    # list of possible moves (up, down, left, right)
    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    for move in moves:
        new_row = empty_space_pos[0] + move[0]
        new_col = empty_space_pos[1] + move[1]
        
        if 0 <= new_row < rows and 0 <= new_col < cols:
            # swap the empty space with the new position
            new_state = state.copy()
            new_state[empty_space_pos], new_state[new_row, new_col] = new_state[new_row, new_col], new_state[empty_space_pos]
            neighbors.append(new_state)
    
    return neighbors

In [29]:
def a_star(start, goal):
    """A* search to solve the puzzle."""
    # Priority queue stores tuples: (f(n), g(n), state, path)
    open_list = []
    start_tuple = tuple(start.flatten())  # Convert start state to tuple
    heapq.heappush(open_list, (0 + manhattan_distance(start, goal), 0, start_tuple, []))  # (f, g, state, path)
    closed_list = set()  # Set to store visited states
    evaluated_cases = 0  # Track the number of cases evaluated
    
    while open_list:
        f, g, current_state_tuple, path = heapq.heappop(open_list)

        # Convert tuple back to numpy array for comparison
        current_state = np.array(current_state_tuple).reshape(goal.shape)

        # np.all() to compare the current state and goal
        if np.all(current_state == goal):
            print("Goal Reached!")
            print(f"Total cost (g): {g}")  
            return path, g, evaluated_cases  

        closed_list.add(current_state_tuple)  # add the current state to visited
        evaluated_cases += 1  # increment the number of evaluated cases
        
        for neighbor in get_neighbors(current_state):
            # convert the neighbor state to a tuple
            neighbor_tuple = tuple(neighbor.flatten())
            
            # skip the state if it has already been visited
            if neighbor_tuple in closed_list:
                continue

            new_g = g + 1  # the cost to reach this neighbor
            new_f = new_g + manhattan_distance(neighbor, goal)  # f(n) = g(n) + h(n)
            
            # append the neighbor to the path as a tuple (for consistency and to avoid numpy ambiguity)
            heapq.heappush(open_list, (new_f, new_g, neighbor_tuple, path + [tuple(neighbor.flatten())]))

    return None, None, evaluated_cases  # No solution

In [ ]:
goal = generate_goal(PUZZLE_DIM)  
start = state 

solution_path, total_cost, evaluated_cases = a_star(start, goal)

if solution_path is not None:
    print("Solution Path:")
    for step in solution_path:
        print(np.array(step).reshape(goal.shape).tolist())  
else:
    print("No solution found")

print(f"Total cost: {total_cost}")

print(f"Total cases evaluated: {evaluated_cases}")


Goal Reached!
Total cost (g): 34
Solution Path:
[[9, 1, 11, 7], [6, 8, 0, 2], [5, 10, 12, 4], [13, 14, 3, 15]]
[[9, 1, 0, 7], [6, 8, 11, 2], [5, 10, 12, 4], [13, 14, 3, 15]]
[[9, 1, 7, 0], [6, 8, 11, 2], [5, 10, 12, 4], [13, 14, 3, 15]]
[[9, 1, 7, 2], [6, 8, 11, 0], [5, 10, 12, 4], [13, 14, 3, 15]]
[[9, 1, 7, 2], [6, 8, 11, 4], [5, 10, 12, 0], [13, 14, 3, 15]]
[[9, 1, 7, 2], [6, 8, 11, 4], [5, 10, 0, 12], [13, 14, 3, 15]]
[[9, 1, 7, 2], [6, 8, 11, 4], [5, 10, 3, 12], [13, 14, 0, 15]]
[[9, 1, 7, 2], [6, 8, 11, 4], [5, 10, 3, 12], [13, 0, 14, 15]]
[[9, 1, 7, 2], [6, 8, 11, 4], [5, 0, 3, 12], [13, 10, 14, 15]]
[[9, 1, 7, 2], [6, 8, 11, 4], [5, 3, 0, 12], [13, 10, 14, 15]]
[[9, 1, 7, 2], [6, 8, 0, 4], [5, 3, 11, 12], [13, 10, 14, 15]]
[[9, 1, 7, 2], [6, 0, 8, 4], [5, 3, 11, 12], [13, 10, 14, 15]]
[[9, 1, 7, 2], [6, 3, 8, 4], [5, 0, 11, 12], [13, 10, 14, 15]]
[[9, 1, 7, 2], [6, 3, 8, 4], [0, 5, 11, 12], [13, 10, 14, 15]]
[[9, 1, 7, 2], [0, 3, 8, 4], [6, 5, 11, 12], [13, 10, 14, 15]]
[[0, 1,